<a href="https://colab.research.google.com/github/pattichis/opt/blob/main/Unconstrained_Optimization_Methods.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Title


## Library imports

In [1]:
import sys
import math
import numpy as np
import unittest

from numpy import dot, transpose, array, diag, linalg, sqrt, identity, zeros, sign, arange

from scipy.stats import uniform

# alltrue is deprecated, use np.all() instead where needed
def alltrue(A):
  return np.all(A)

# matrixmultiply() is deprecated
def matrixmultiply(A, B):
  return np.dot(A, B) # Performs dot product or matrix multiplication


In [2]:
def MatQuad(l, M, r):
	"""
	In several instances the product $l^T M r$ appears
	and so we define it here to simplify the
	implementation of various procedures.

	Inputs:
		l is an n-dimensional vector
		M is an nxn matrix
		r is an n-dimensional vector

	Outputs:
		returns the product $l^TMr$
	"""
	return matrixmultiply(matrixmultiply(transpose(l), M), r)

# Objective Functions

## Base class for objective functions

In [3]:
class ObjectiveFunction:
	"""
	This is the base class for the objective functions that we
	will evaluate for optimization.  This class ensures that a function,
	its exact gradient and its exact Hessian are grouped into a single,
	logical unit.

	Furthermore, this base class tracks the number of unique function
	evaluations required on a given optimization run.

	It does this by storing the supplied inputs $x$ to each function
	in an array.  If the input $x$ is already in the array, it is
	not added again.  Therefore, the number of unique function
	evaluations is the size of the array.
	"""
	def __init__(self, ninit = 0):
		self.f_cache = []
		self.f_grad_cache = []
		self.f_hessian_cache = []
		self.n = ninit

	def __check_n(self, x):
		"""
		The first time an x is supplied to either real_f, grad_f,
		or hessian_f, we extract the dimension of this x and set
		our internal n value to it.

		Or, for functions of fixed dimesions, they may set n
		in their constructor.


		This function either sets the n value if it hasn't been set,
		or compares the dimension of the x supplied to our n
		value to make sure that an x of appropriate dimension
		has been supplied.
		"""
		if self.n:
			if x.shape[0] != self.n:
				raise 'bad dimension on x'
		else:
			self.n = x.shape[0]


	def descriptive_name(self):
		"""
		Return a descriptive name of the objective function, for
		use in the table headers.
		"""
		return 'generic objective function'

	def eval_count(self):
		"""
		Returns the number of times f, grad_f, and hessian_f have
		been called with unique inputs.
		"""
		return len(self.f_cache) + len(self.f_grad_cache) + \
			len(self.f_hessian_cache)

	def real_f(self, x):
		"""
		The actual function you want to override in a subclass.

		Should return the value of the function evaluated at $x$.
		"""
		return 0

	def f(self, x):
		"""
		Returns the value of the function evaluated at $x$.

		Uses caching to count the number of unique function
		evaluations.

		This is the function that the optimization methods
		should call.
		"""
		self.__check_n(x)

		for cache_x in self.f_cache:
			# x == cache_x on array types returns a matrix
			# of value-by-value comparisons.  The alltrue
			# numpy function returns true iff all comparisons
			# returned true
			if alltrue(cache_x == x):
				return self.real_f(x)

		self.f_cache.append(x)

		return self.real_f(x)

	def real_grad_f(self, x):
		"""
		The actual functino you want to override in a subclass.

		Should return the value of the exact gradient of the
		function $f$ evaluated at $x$.
		"""
		return 0

	def grad_f(self, x):
		"""
		Returns the value of the gradient of the function f
		evaluated at $x$.
		"""
		self.__check_n(x)

		for cache_x in self.f_grad_cache:
			if alltrue(cache_x == x):
				return self.real_grad_f(x)

		self.f_grad_cache.append(x)

		return self.real_grad_f(x)

	def real_hessian_f(self, x):
		"""
		The actual function you want to override in a subclass.

		Returns the value of the exact Hessian of the function
		$f$ evaluated at $x$.
		"""
		return 0

	def hessian_f(self, x):
		"""
		Returns the value of the Hessian of the function f
		evaluated at $x$.
		"""
		self.__check_n(x)

		for cache_x in self.f_hessian_cache:
			if alltrue(cache_x == x):
				return self.real_hessian_f(x)

		self.f_hessian_cache.append(x)

		return self.real_hessian_f(x)


## Rosenbrock

In [4]:

class Rosenbrock(ObjectiveFunction):
	"""
	The Rosenbrock function is defined in Nocedal and Wright's
	textbook Numerical Optimization, Formula 2.23, on page 30.

	The formula has been rewritten slightly to account for the fact
	that the input array has two elements at indicies 0 and 1, whereas
	the formula defines two elements $x_1$ and $x_2$.  The reader should
	consider code $x_0$ = book $x_1$ and code $x_1$ = book $x_2$.
	Therefore we write it as:

	$f(x) = 100(x_1 - x_0^2)^2 + (1 - x_0)^2$
	"""
	def __init__(self):
		ObjectiveFunction.__init__(self)
		self.n = 2

	def descriptive_name(self):
		"""
		Return a descriptive name of the objective function, for
		use in the table headers.
		"""
		return 'the Rosenbrock function'

	def real_f(self, x):
		"""
		Returns the value of Rosenbrock function evaluated at $x$.

		Inputs:
			$x$ is a 2D Numeric array of floating point numbers which
			represent the inputs to the Rosenbrock function.

		Outputs:
			Returns a floating point scalar with the value of the
			Rosenbrock function evaluated at $x$.

		Side Effects:
			None.
		"""
		return (100 * (x[1] - x[0]**2)**2) + (1 - x[0])**2

	def real_grad_f(self, x):
		"""
		Returns the value of the gradient of the Rosenbrock function
		evaluated at $x$.

		The gradient of the Rosenbrock function was calculated by hand
		to be:

		$\nabla f(x) = \left[ \begin{array}{c}
		400x_0^3 - 400x_1x_0 + 2x_0 - 2 \\
		-200x_0^2 + 200x_1 \end{array} \right]$

		Inputs:
            		$x$ is a 2D Numeric array of floating point numbers
				which represent the inputs to the Rosenbrock
				gradient function

		Outputs:
			Returns a 2D Numeric array with the values of the
			gradient of the Rosenbrock function evaluated at $x$.

		Side Effects:
			None.
	    """
		return array((400*x[0]**3 - 400*x[1]*x[0] + 2*x[0] - 2,
					  -200*x[0]**2 + 200*x[1]))

	def real_hessian_f(self, x):
		"""
		Returns the value of the Hessian of the Rosenbrock function
		evaluated at $x$.

		The Hessian of the Rosenbrock function was calculated by hand
		to be:

		$\nabla^2 f(x) = \left[ \begin{array}{cc}
		1200x_0^2 - 400x_1 + 2  & -400x_0 \\
		-400x_0 & 200 \end{array} \right]$

		Inputs:
			$x$ is a 2D Numeric array of the floating point numbers
			which represent the inputs to the Rosenbrock Hessian
			function.

		Outputs:
			Returns a 2x2 Numeric matrix with the values of
			the Hessian of the Rosenbrock function evaluated at $x$.

		Side Effects:
			None.
		"""
		return array(([1200*x[0]**2 - 400*x[1] + 2, -400*x[0]],
					  [-400*x[0], 200]))


<>:50: SyntaxWarning: invalid escape sequence '\l'
<>:77: SyntaxWarning: invalid escape sequence '\l'
<>:50: SyntaxWarning: invalid escape sequence '\l'
<>:77: SyntaxWarning: invalid escape sequence '\l'
/tmp/ipython-input-1940060607.py:50: SyntaxWarning: invalid escape sequence '\l'
  $\nabla f(x) = \left[ \begin{array}{c}
/tmp/ipython-input-1940060607.py:77: SyntaxWarning: invalid escape sequence '\l'
  $\nabla^2 f(x) = \left[ \begin{array}{cc}


## Problem 4.1 and 4.3 functions

In [5]:
class Problem4_1(ObjectiveFunction):
	"""
	This class is an implementation of the function defined in
	Problem 4.1 of Nocedal and Wright, on page 97.
	"""
	def descriptive_name(self):
		return 'the function defined in problem 4.1'

	def real_f(self, x):
		"""
		Returns the value of the function evaluated at $x$.

		The function is defined to be:

		$f(x) = 10(x_2 - x_1^2)^2 + (1 - x_1)^2$
       		"""
		return 10*(x[1] - x[0]**2)**2 + (1 - x[0])**2

	def grad_f(self, x):
		"""
		Returns the value of the gradient of the function,
		$\nabla f$ evaluated at $x$.

		The gradient is defined to be:

		$\nabla f(x) = \left[ \begin{array}{c}
		  -40x_1x_2 + 40x_1 - 2 + 2x_1 \\ 20x_2 - 20x_1^2 \end{array} \right]$
		"""
		return array((-40.0*x[0]*x[1] + 40*x[0] - 2 + 2*x[0],
			      20*x[1] - 20*x[0]))

	def hessian_f(self, x):
		"""
		Returns the value of the Hessian of the function,
		$\nabla^2 f$ evaluated at $x$.

		The Hessian is defined to be:

		$\nabla^2 f(x) = \left[ \begin{array}{cc}
		  -40x_2 + 42 & -40x_1 \\ -40x_1 & 20 \end{array} \right]$
		"""
		return array(([-40*x[1]+42, -40*x[0]], [-40*x[0], 20]))


class Problem4_3(ObjectiveFunction):
	"""
	This class is an implementation of the function defined in
	Problem 4.3 of Nocedal and Wright, on page 97.
	"""

	def descriptive_name(self):
		return 'the function defined in problem 4.3, n = %d' % self.n

	def real_f(self, x):
		"""
		Returns the value of the function evaluated at x.  Note that the
		dimension on x is taken to be twice the value of the $n$ in the
		summation, as that is the smallest size vector that could be
		processed in this summation.

		The function is defined to be:

		$f(x) = \sum_{i=1}^n \left[(1-x_{2i-1})^2 + 10(x_{2i} - x_{2i-1}^2)^2 \right]$
		"""
		accum = 0.0

		for i in range(0, self.n//2):
			accum += (1 - x[2*i])**2 + 10*(x[2*i+1] - x[2*i]**2)**2

		return accum

	def real_grad_f(self, x):
		"""
		Returns the value of the gradient of the function evaluated at x.
		The $i$th component of the gradient, for $i$ valid from $0$ to
		$2n -1$ was calculated by hand to be:

		$\nabla f_i(x) = \left\{ \begin{array}{rcl}
			-2 + 2x_i - 40x_ix_{i+1} + 40x_i^3 & \mbox{for} & i \mbox{ even} \\
			20x_i - 20x_{i-1}^2                & \mbox{for} & i \mbox{ odd} \end{array} \right.$
		"""
		grad_transpose = []

		for i in range(0, self.n):
			if not i % 2:
				# i even
				grad_transpose.append(-2 + 2*x[i] \
				  - 40*x[i+1]*x[i] + 40*x[i]**3)
			else:
				# i odd
				grad_transpose.append(20*x[i] - 20*x[i-1]**2)

		return transpose(array((grad_transpose)))

	def real_hessian_f(self, x):
		"""
		Returns the hessian of the function evaluated at $x$.

		Just constructs the diagonal terms, then uses numpy's
		diag() operator to construct an nxn matrix with hessian
		as the diagonal members of that matrix.

		The $ij$ compmonent of the hessian was calculated by
		hand to be:

		$\nabla^2 f_{i,j}(x) = \left\{ \begin{array}{rcll}
			2 - 40x_{i+1} + 120x_i^2 & \mbox{for} & i=j & i \mbox{ even} \\
			20 & \mbox{for} & i=j & i \mbox{ odd} \\
			0 & & & \mbox{otherwise} \end{array} \right.$
		"""
		hessian = []

		for i in range(0, self.n):
			if not i % 2:
				hessian.append(2.0 - 40*x[i+1] + 120*x[i]**2)
			else:
				hessian.append(20.0)

		return diag(array((hessian)))


<>:26: SyntaxWarning: invalid escape sequence '\l'
<>:39: SyntaxWarning: invalid escape sequence '\l'
<>:63: SyntaxWarning: invalid escape sequence '\s'
<>:78: SyntaxWarning: invalid escape sequence '\l'
<>:106: SyntaxWarning: invalid escape sequence '\l'
<>:26: SyntaxWarning: invalid escape sequence '\l'
<>:39: SyntaxWarning: invalid escape sequence '\l'
<>:63: SyntaxWarning: invalid escape sequence '\s'
<>:78: SyntaxWarning: invalid escape sequence '\l'
<>:106: SyntaxWarning: invalid escape sequence '\l'
/tmp/ipython-input-1870734557.py:26: SyntaxWarning: invalid escape sequence '\l'
  $\nabla f(x) = \left[ \begin{array}{c}
/tmp/ipython-input-1870734557.py:39: SyntaxWarning: invalid escape sequence '\l'
  $\nabla^2 f(x) = \left[ \begin{array}{cc}
/tmp/ipython-input-1870734557.py:63: SyntaxWarning: invalid escape sequence '\s'
  $f(x) = \sum_{i=1}^n \left[(1-x_{2i-1})^2 + 10(x_{2i} - x_{2i-1}^2)^2 \right]$
/tmp/ipython-input-1870734557.py:78: SyntaxWarning: invalid escape sequence '\l

## Objective functions for verifying line search

In [6]:
class testObjFunc(ObjectiveFunction):
	"""
	A simple test Objective Function, with root
	at $x^\ast = (0,0)$, for testing the Line Search
	methods.

	$f(x) = x_0^2 + x_1^2$
	"""
	def descriptive_name(self):
		return 'test function'

	def real_f(self, x):
		return x[0]**2 + x[1]**2

	def real_grad_f(self, x):
		"""
		The gradient $\nabla f$ is calculated to be:

		$\nabla f = \left[ \begin{array}{c}
		   2x_0 \\ 2x_1 \end{array} \right]$
		"""
		return array((2.0*x[0], 2.0*x[1]))

	def real_hessian_f(self, x):
		"""
		The hessian $\nabla^2 f$ is calculated to be:

		$\nabla^2 f = \left[ \begin{array}{cc}
		  2 & 0 \\ 0 & 2 \end{array} \right]$
		"""
		return array(([2.0, 0.0], [0.0, 2.0]))

class testSecondFunc(ObjectiveFunction):
	"""
	A simple test objective function, with root at
	$x^\ast = (1,1)$ for testing the line search methods.
	This one is a bit more ill conditioned for a line search
	in that the variables do not contribute equally to
	the solution of the problem.

	$f(x) = 250.0(x_0 - 1)^2 + 0.025*(x_1 - 1)^2$
	"""
	def descriptive_name(self):
		return 'a second test function'

	def real_f(self, x):
		"""
		Returns the function evaluated at $x$
		"""
		return 250.0*(x[0] - 1.0)**2 + 0.025*(x[1] - 1.0)**2

	def real_grad_f(self, x):
		"""
		The gradient $\nabla f$ is calculated to be:

		$\nabla f = \left[ \begin{array}{c}
		  500x_0 - 500 \\ 0.05x_1 - 0.05 \end{array} \right]$
		"""
		return array((500*x[0] - 500, 0.05*x[1] - 0.05))

	def real_hessian_f(self, x):
		"""
		The hessian $\nabla^2 f$ is calculated to be:

		$\nabla^2 f = \left[ \begin{array}{cc}
		  500 & 0 \\ 0 & 0.05 \end{array} \right]$
		"""
		return array(([500, 0.0], [0.0, 0.05]))



<>:19: SyntaxWarning: invalid escape sequence '\l'
<>:28: SyntaxWarning: invalid escape sequence '\l'
<>:56: SyntaxWarning: invalid escape sequence '\l'
<>:65: SyntaxWarning: invalid escape sequence '\l'
<>:19: SyntaxWarning: invalid escape sequence '\l'
<>:28: SyntaxWarning: invalid escape sequence '\l'
<>:56: SyntaxWarning: invalid escape sequence '\l'
<>:65: SyntaxWarning: invalid escape sequence '\l'
/tmp/ipython-input-1420064506.py:19: SyntaxWarning: invalid escape sequence '\l'
  $\nabla f = \left[ \begin{array}{c}
/tmp/ipython-input-1420064506.py:28: SyntaxWarning: invalid escape sequence '\l'
  $\nabla^2 f = \left[ \begin{array}{cc}
/tmp/ipython-input-1420064506.py:56: SyntaxWarning: invalid escape sequence '\l'
  $\nabla f = \left[ \begin{array}{c}
/tmp/ipython-input-1420064506.py:65: SyntaxWarning: invalid escape sequence '\l'
  $\nabla^2 f = \left[ \begin{array}{cc}


## Unit tests for objective functions

In [7]:
#####################
# unit testing code #
#####################
import unittest
import numpy as np # Add numpy import explicitly

class ObjectiveFunctionTestCase(unittest.TestCase):
        def testFunctionCaching(self):
                """
                Check that the ObjectiveFunction class correctly
                counts the number of unique inputs to f
                """
                o = ObjectiveFunction()

                assert len(o.f_cache) == 0, \
                        'incorrect starting count for unique inputs'
                o.f(np.array((0.0, 0.0)))
                assert len(o.f_cache) == 1, \
                        'incorrect count for one input'
                o.f(np.array((0.0, 0.0)))
                assert len(o.f_cache) == 1, \
                        'incorrect count for one unique input'
                o.f(np.array((1.0, 0.0)))
                assert len(o.f_cache) == 2, \
                        'incorrect count for two unique inputs'

                # check that the f cache didn't end up corrupting
                # the other caches
                assert len(o.f_grad_cache) == 0, \
                        'f cache corrupted gradient cache'
                assert len(o.f_hessian_cache) == 0, \
                        'f cache corrupted hessian cache'

                # check gradient cache
                o.grad_f(np.array((1.0, 0.0)))
                assert len(o.f_grad_cache) == 1, \
                        'f grad cache not storing inputs'
                o.grad_f(np.array((1.0, 0.0)))
                assert len(o.f_grad_cache) == 1, \
                        'f grad cache not storing unique inputs'

                # check Hessian cache
                o.hessian_f(np.array((0.0, 1.0)))
                assert len(o.f_hessian_cache) == 1, \
                        'f hessian cache not storing inputs'
                o.hessian_f(np.array((0.0, 1.0)))
                assert len(o.f_hessian_cache) == 1, \
                        'f hessian cache not storing unique inputs'

class RosenbrockTestCase(unittest.TestCase):
        def testRosenbrock(self):
                """
                Check that the Rosenbrock function returns correct
                output for certain inputs
                """
                r = Rosenbrock()

                assert r.f(np.array((0.0, 0.0))) == 1, \
                        'incorrect value returned for x=[0,0]'
                assert r.f(np.array((1.0, 0.0))) == 100, \
                        'incorrect value returned for x=[1,0]'
                assert r.f(np.array((0.0, 1.0))) == 101, \
                        'incorrect value returned for x=[0,1]'
                # check the case of the minimum of the Rosenbrock at
                # $x^\ast = (1,1)$
                assert r.f(np.array((1.0, 1.0))) == 0, \
                        'incorrect value returned for x=[1,1]'


        def testRosenbrockGradient(self):
                """
                Check that the Rosenbrock gradient function
                returns correct output for certain inputs
                """
                r = Rosenbrock()

                assert np.all(r.grad_f(np.array((0.0, 0.0)))\
                               == np.array((-2.0, 0.0))), \
                        'incorrect value returned for x=[0,0]'
                assert np.all(r.grad_f(np.array((1.0, 0.0))) \
                               == np.array((400.0, -200.0))), \
                        'incorrect value returned for x=[1,0]'
                assert np.all(r.grad_f(np.array((0.0, 1.0))) \
                               == np.array((-2.0, 200.0))), \
                        'incorrect value returned for x=[0,1]'
                # gradient should be zero at the minimum of the Rosenbrock at
                # $x^\ast = (1,1)$
                assert np.all(r.grad_f(np.array((1.0, 1.0))) == \
                               np.array((0.0, 0.0))), \
                        'incorrect value returned for x=[1,1]'

        def testRosenbrockHessian(self):
                """
                Check that the Rosenbrock Hessian function
                returns correct output for certain inputs
                """
                r = Rosenbrock()
                assert np.all(r.hessian_f(np.array((0.0, 0.0))) \
                               == np.array(([2, 0],\
                        [0, 200]))), 'incorrect value returned for x=[0,0]'
                assert np.all(r.hessian_f(np.array((1.0, 0.0))) \
                               == np.array(([1202, -400],\
                        [-400, 200]))), 'incorrect value returned for x=[1,0]'
                assert np.all(r.hessian_f(np.array((0.0, 1.0))) \
                               == np.array(([-398, 0],\
                        [0, 200]))), 'incorrect value returned for x=[0,1]'
                # Hessian should be pos def at the minimum of the Rosenbrock
                # at $x^\ast= (1,1)$
                assert np.all(r.hessian_f(np.array((1.0, 1.0))) \
                               == np.array(([802, -400],\
                        [-400, 200]))), 'incorrect value returned for x=[1,1]'

class Problem4_1TestCase(unittest.TestCase):
        def testProblem4_1(self):
                """
                Check that the Problem4_1 function returns correct
                output for certain inputs
                """
                p = Problem4_1()

                print(p.f(np.array((0.0, 0.0))))

                assert p.f(np.array((0.0, 0.0))) == 1, \
                        'incorrect value returned for x=[0,0]'
                assert p.f(np.array((1.0, 0.0))) == 10, \
                        'incorrect value returned for x=[1,0]'
                assert p.f(np.array((0.0, 1.0))) == 11, \
                        'incorrect value returned for x=[0,1]'
                # check the case of the minimum of Problem4_1 at
                # $x^\ast = (1,1)$
                assert p.f(np.array((1.0, 1.0))) == 0, \
                        'incorrect value returned for x=[1,1]'


        def testProblem4_1Gradient(self):
                """
                Check that the Problem4_1 gradient function
                returns correct output for certain inputs
                """
                p = Problem4_1()

                assert np.all(p.grad_f(np.array((0.0, 0.0))) == np.array((-2.0, 0.0))), \
                        'incorrect value returned for x=[0,0]'
                assert np.all(p.grad_f(np.array((1.0, 0.0))) == np.array((40.0, -20.0))), \
                        'incorrect value returned for x=[1,0]'
                assert np.all(p.grad_f(np.array((0.0, 1.0))) == np.array((-2.0, 20.0))), \
                        'incorrect value returned for x=[0,1]'
                # gradient should be zero at the minimum of the Problem4_1 at
                # $x^\ast = (1,1)$
                assert np.all(p.grad_f(np.array((1.0, 1.0))) == np.array((0.0, 0.0))), \
                        'incorrect value returned for x=[1,1]'

        def testProblem4_1Hessian(self):
                """
                Check that the Problem4_1 Hessian function
                returns correct output for certain inputs
                """
                p = Problem4_1()
                print("\n--- Debugging testProblem4_1Hessian ---")

                x_test = np.array((0.0, 0.0))
                calculated_hessian = p.hessian_f(x_test)
                expected_hessian = np.array(([42, 0],[0, 20]))
                print(f"Input x: {x_test}")
                print(f"Calculated Hessian: {calculated_hessian}")
                print(f"Expected Hessian: {expected_hessian}")
                assert np.all(calculated_hessian == expected_hessian), f"Calculated Hessian: {calculated_hessian} at x=[0, 0]"

                x_test = np.array((1.0, 0.0))
                calculated_hessian = p.hessian_f(x_test)
                expected_hessian = np.array(([42, -40],[-40, 20]))
                print(f"Input x: {x_test}")
                print(f"Calculated Hessian: {calculated_hessian}")
                print(f"Expected Hessian: {expected_hessian}")
                assert np.all(calculated_hessian == expected_hessian), 'incorrect value returned for x=[1,0]'

                x_test = np.array((0.0, 1.0))
                calculated_hessian = p.hessian_f(x_test)
                expected_hessian = np.array(([2, 0],[0, 20]))
                print(f"Input x: {x_test}")
                print(f"Calculated Hessian: {calculated_hessian}")
                print(f"Expected Hessian: {expected_hessian}")
                assert np.all(calculated_hessian == expected_hessian), 'incorrect value returned for x=[0,1]'

                # Hessian should be pos def at the minimum of the Problem4_1
                # at $x^\ast= (1,1)$
                x_test = np.array((1.0, 1.0))
                calculated_hessian = p.hessian_f(x_test)
                expected_hessian = np.array(([2, -40],[-40, 20]))
                print(f"Input x: {x_test}")
                print(f"Calculated Hessian: {calculated_hessian}")
                print(f"Expected Hessian: {expected_hessian}")
                assert np.all(calculated_hessian == expected_hessian), 'incorrect value returned for x=[1,1]'


class Problem4_3TestCase(unittest.TestCase):
        def testFunctionValues(self):
                pf = Problem4_3()

                assert pf.f(np.array((0.0, 0.0))) == 1, \
                        'incorrect value returned for x=[0,0]'
                assert pf.f(np.array((1.0, 0.0))) == 10, \
                        'incorrect value returned for x=[1,0]'
                assert pf.f(np.array((0.0, 1.0))) == 11, \
                        'incorrect value returned for x=[0,1]'
                assert pf.f(np.array((1.0, 1.0))) == 0, \
                        'incorrect value returned for x=[1,1]'

        def testFunctionGradientValues(self):
                pf = Problem4_3()

                assert np.all(pf.grad_f(np.array((0.0, 0.0))) == np.array((-2.0, 0.0))), \
                        'incorrect value returned for x=[0,0]'
                assert np.all(pf.grad_f(np.array((1.0, 0.0))) == np.array((40.0, -20.0))), \
                        'incorrect value returned for x=[1,0]'
                assert np.all(pf.grad_f(np.array((0.0, 1.0))) == np.array((-2.0, 20.0))), \
                        'incorrect value returned for x=[0,1]'
                assert np.all(pf.grad_f(np.array((1.0, 1.0))) == np.array((0.0, 0.0))), \
                        'incorrect value returned for x=[1,1]'

        def testFunctionHessianValues(self):
                pf = Problem4_3()

                assert np.all(pf.hessian_f(np.array((0.0, 0.0))) == np.array(([2.0, 0.0],\
                                                                         [0.0, 20.0]))), \
                        'incorrect value returned for x=[0,0]'
                assert np.all(pf.hessian_f(np.array((1.0, 0.0))) == np.array(([122.0, 0.0], \
                                                                        [0.0, 20.0]))), \
                         'incorrect value returned for x=[1,0]'
                assert np.all(pf.hessian_f(np.array((0.0, 1.0))) == np.array(([-38.0, 0.0], \
                                                                         [0.0, 20]))), \
                         f"incorrect value returned for x=[0,1] {pf.hessian_f(np.array((0.0, 1.0)))}"

                assert np.all(pf.hessian_f(np.array((1.0, 1.0))) == np.array(([82.0, 0.0], \
                                                                        [0.0, 20.0]))), \
                         f"incorrect value returned for x=[1,1] {pf.hessian_f(np.array((1.0, 1.0)))}"

                pf4 = Problem4_3()
                assert np.all(pf4.hessian_f(np.array((1.0, 1.0, 1.0, 1.0))) == \
                        np.array(([82.0, 0.0,  0.0,  0.0],
                                [0.0, 20.0,  0.0,  0.0],
                                [0.0,  0.0, 82.0,  0.0],
                                [0.0,  0.0,  0.0, 20.0]))), \
                         'incorrect value returned for x=[1,1,1,1]'

In [8]:
unittest.main(argv=['first-arg-is-ignored'], exit=False)

..........
----------------------------------------------------------------------
Ran 10 tests in 0.010s

OK


1.0

--- Debugging testProblem4_1Hessian ---
Input x: [0. 0.]
Calculated Hessian: [[42. -0.]
 [-0. 20.]]
Expected Hessian: [[42  0]
 [ 0 20]]
Input x: [1. 0.]
Calculated Hessian: [[ 42. -40.]
 [-40.  20.]]
Expected Hessian: [[ 42 -40]
 [-40  20]]
Input x: [0. 1.]
Calculated Hessian: [[ 2. -0.]
 [-0. 20.]]
Expected Hessian: [[ 2  0]
 [ 0 20]]
Input x: [1. 1.]
Calculated Hessian: [[  2. -40.]
 [-40.  20.]]
Expected Hessian: [[  2 -40]
 [-40  20]]


# Line Search Algorithms

## BacktrackingAlpha(.): Find alpha to satisfy linear approximation

In [9]:

def BacktrackingAlpha(fobj, x_k, p_k, alpha_bar):
	"""
	Uses the backtracking procedure defined on page 41
	of Nocedal and Wright to find the $\alpha_k$ to
	be used for an iteration of a line search
	algorithm.

	Inputs:
		fobj is a function object from ObjectiveFunctions.py

		x_k is the $x_k$ point that we are finding $\alpha$ in

		p_k is the $p_k$ search direction

		alpha_bar is the $\overline{\alpha}$, the initial value for $\alpha$

	Outputs:
		The $\alpha$ value to use for this iteration
	"""
	# $\rho$ and $c$ are constants that we set here
	rho = 0.5
	c = 1e-4

	# $\alpha \leftarrow \overline{\alpha}$
	alpha = alpha_bar
	# repeat until
	# $f(x_k + \alpha p_k) \leq f(x_k) + c \alpha \nabla f_k^T p_k$
	while fobj.f(x_k + alpha * p_k) > fobj.f(x_k) + \
			c * alpha * matrixmultiply(\
		transpose(fobj.grad_f(x_k)), p_k):
		# $\alpha \leftarrow \rho \alpha$
		alpha = rho * alpha

	return alpha


<>:15: SyntaxWarning: invalid escape sequence '\o'
<>:15: SyntaxWarning: invalid escape sequence '\o'
/tmp/ipython-input-2396795012.py:15: SyntaxWarning: invalid escape sequence '\o'
  alpha_bar is the $\overline{\alpha}$, the initial value for $\alpha$


## Newtons Method Line Search

In [10]:
def NewtonsMethodLineSearch(x_0, alpha_0, f_alpha_k, fobj, xtol):
	"""
	Finds a point within a small distance from a stationary point
	of the function f using the basic line search algorithm defined
	in chapter 3 of Nocedal and Wright's Numerical Optimization, where

	$x_{k+1} = x_k + \alpha_k p_k$

	For Newton's Method, the $B_k = \nabla^2f(x_k)$, so

	$p_k = -\nabla^2f(x_k)\nabla f(x_k)$

	This function supports arbitrary line search procedures
	to find the $\alpha_k$.

	Inputs:
		x_0 is a Numeric array giving the value of $x_0$.

		alpha_0 is a floating point scalar giving the value of $\alpha_0$.

		f_alpha_k is a function reference, a function that takes an
			$x_k$, a $p_k$, and an $\overline{\alpha}$ and returns
			an $alpha_k$.  For reference see BacktrackingAlpha above.

		fobj is an ObjectiveFunction object.  The functions must take
			inputs of the same degree as x_0.

		xtol is the tolerance on the stationary point.  The
			algorithm will terminate when $\|\nabla f(x_k)\| \leq$ xtol.

	Outputs:
		Returns the computed value of the stationary point of $f$, $x^\ast$.

	Side Effects:
		Writes the values of $x_k$, $\alpha_k$, and $p_k$ at each iteration
		to stdout.
	"""
	x_k = x_0
	iterations = 0

	# print out our table header
	print ('\n\nNewton\'s Method Line Search on %s' \
		% fobj.descriptive_name())
	print ('alpha_0 = %f, x_0 = %s' % (alpha_0, x_0))
	print('{:10s}{:10s}{:10s}{:10s}'.format('iteration', 'alpha_k', 'x_k', 'f evals'))

	# repeat loop until $\|\nabla f(x_k)\| \leq$ xtol
	while dot(fobj.grad_f(x_k), fobj.grad_f(x_k)) > xtol:
		# $p_k = -B_k^{-1}\nabla f_k = -\nabla^2 f(x_k) \nabla f_k$
		p_k = -1.0*matrixmultiply(linalg.inv(fobj.hessian_f(x_k)), \
							  fobj.grad_f(x_k))

    # apply the line search technique supplied
		alpha_k = f_alpha_k(fobj, x_k, p_k, alpha_0)

    # $x_{k+1} = x_k + \alpha_k p_k$
		x_k = x_k + (alpha_k * p_k)
		iterations += 1

		# print out the line of the table
		print(f"{iterations:10d}{alpha_k:10.4f}{x_k}{fobj.eval_count():10d}")

		# make sure that this line gets written to stdout
		sys.stdout.flush()

	return x_k



<>:22: SyntaxWarning: invalid escape sequence '\o'
<>:22: SyntaxWarning: invalid escape sequence '\o'
/tmp/ipython-input-3677109062.py:22: SyntaxWarning: invalid escape sequence '\o'
  $x_k$, a $p_k$, and an $\overline{\alpha}$ and returns


## Steepest Descent Line Search

In [11]:
def SteepestDescentLineSearch(x_0, alpha_0, f_alpha_k, fobj, xtol):
	"""
	Uses the steepest decent line search, in which $B_k = I$,
	to find the minimum value of the objective function.

	Inputs:
		x_0 is the Numeric array giving the value of $x_0$

		alpha_0 is a floating point scalar giving the value of $\alpha_0$

		f_alpha_k is a function reference, a function that takes an
			$x_k$, a $p_k$, and an $\overline{\alpha}$ and returns
			an $alpha_k$.  For reference see BacktrackingAlpha above.


		fobj is an ObjectiveFunction object.  The functions must take
			inputs of the same degree as x_0.

		xtol is the tolerance on the stationary point.  The
			algorithm will terminate when $\|\nabla f(x_k)\|^2 \leq$ xtol.

	Outputs:
		Returns the computed value of the stationary point of $f$, $x^\ast$
		within tolerance xtol.

	Side Effects:
		Writes the values of $x_k$, $\alpha_k$, and $p_k$ at each iteration
		to stdout.
	"""
	x_k = x_0
	iterations = 0

	# print out our table header
	print ('\n\nSteepest Descent Line Search on %s' \
		% fobj.descriptive_name())
	print ('alpha_0 = %f, x_0 = %s' % (alpha_0, x_0))
	print ('iteration\talpha_k\t\tx_k\t\t\t\tf evals')

	# repeat loop until $\|\nabla f(x_k)\| \leq$ xtol
	while dot(fobj.grad_f(x_k), fobj.grad_f(x_k)) > xtol:
		# $p_k = -B_k^{-1}\nabla f_k = -\nabla f_k$
		p_k = -1.0 * fobj.grad_f(x_k)

		# apply the line search technique supplied
		alpha_k = f_alpha_k(fobj, x_k, p_k, alpha_0)

		# $x_{k+1} = x_k + \alpha_k p_k$
		x_k = x_k + (alpha_k * p_k)
		iterations += 1

		# print out a line of the table
		print ('%d\t\t%f\t%s\t%d' \
			% (iterations, alpha_k, x_k, fobj.eval_count()))

		# make sure that this line gets written to stdout
		sys.stdout.flush()

	return x_k


<>:12: SyntaxWarning: invalid escape sequence '\o'
<>:12: SyntaxWarning: invalid escape sequence '\o'
/tmp/ipython-input-2888884521.py:12: SyntaxWarning: invalid escape sequence '\o'
  $x_k$, a $p_k$, and an $\overline{\alpha}$ and returns


## zoom(.) Strong Wolfe Conditions

In [12]:
def zoom(fobj, x_k, p_k, alpha_lo, alpha_hi, phi_0, phi_prime_0, c_1, c_2):
	"""
	This is an implementation of Algorithm 3.3, page 60 of Nocedal and Wrigt
	which is a subprocedure to implement the Strong Wolfe Conditions line
	search provided below in the Strong Wolfe function.

	Inputs:
		fobj is an objective function object

		x_k is the $x_k$ value, a Numeric array, used to find $\alpha$

		p_k is the $p_k$ value, a Numeric array, used to find $\alpha$

		alpha_lo is the $\alpha_{lo}$ value, a floating point scalar

		alpha_hi is the $\alpha_{hi}$ value, a floating point scalar

		phi_0 is $\phi(0)$, a floating point scalar

		phi_prime_0 is $\phi'(0)$, a floating point scalar

		c_1 is the constant $c_1$ from the Strong Wolfe conditions

		c_2 is the constant $c_2$ from the Strong Wolfe conditions

	Returns:
		A suitable value for $\alpha$ that meets the Strong Wolfe
			conditions, a floating point scalar.

	Side Effects:
		None
	"""

	iteration = 1
	max_iterations = 100
	while (iteration <= max_iterations):
		iteration += 1

		# interpolate using bisection to find a trial step
		# length $\alpha_j$ between $\alpha_{lo}$ and $\alpha_{hi}$
		alpha_j = (alpha_hi + alpha_lo) / 2.0

		# on the rare occasion that $\alpha_{hi} = \alpha_{lo}$ we actually go outside that range to find a usable value.
		if alpha_hi == alpha_lo:
			alpha_j = alpha_hi / 2.0

		# Evaluate $\phi(\alpha_j)$
		phi_alpha_j = fobj.f(x_k + alpha_j*p_k)

		# if $\phi(\alpha_j) > \phi(0) + c_1\alpha_j\phi'(0)$ or $\phi(\alpha_j) \geq \phi(\alpha_{lo})$
		if phi_alpha_j > phi_0 + c_1*alpha_j*phi_prime_0 or phi_alpha_j >= fobj.f(x_k + alpha_lo*p_k):
			# $\alpha_{hi} \leftarrow \alpha_j$
			alpha_hi = alpha_j
		else:
			# Evaluate $\phi'(\alpha_j)$
			phi_prime_alpha_j = dot(fobj.grad_f(x_k + alpha_j*p_k), p_k)

			# if $|\phi'(\alpha_j)| \leq -c_2 \phi'(0)$
			if abs(phi_prime_alpha_j) <= -c_2*phi_prime_0:
				# set $\alpha_\ast \leftarrow \alpha_j$ and stop
				return alpha_j

			# if $\phi'(\alpha_j)(\alpha_{hi} - \alpha_{lo}) \geq 0$
			if phi_prime_alpha_j*(alpha_hi - alpha_lo) >= 0:
				# $\alpha_{hi} \leftarrow \alpha_{lo}$
				alpha_hi = alpha_lo

			# $\alpha_{lo} \leftarrow \alpha_j$
			alpha_lo = alpha_j

		print("zoom: max iterations exited!")
		quit()



def StrongWolfe(fobj, x_k, p_k, alpha_max):
	"""
	This is an implementation of Algorithm 3.2, page 59 of Nocedal and Wright,
	a line search algorithm that meets the Strong Wolfe conditions.

	Inputs:
		fobj is an objective function object

		x_k is the $x_k$ value, the staring point to be used for the
			search for $\alpha$, and is a Numeric array

		p_k is the $p_k$ value, the search direction to be used for the
			search for $\alpha$, and is a Numeric array

		alpha_max is some maximum value that the algorithm will not exceed.

	Returns:
		A suitable value for $\alpha$, a floating point scalar, that
			satisfiies the Strong Wolfe Conditions

	Side Effects:
		None
	"""
	# set $\alpha_0 \leftarrow 0$, choose $\alpha_1 > 0$ and $\alpha_{max}$
	alpha_i_minus_1 = 0
	alpha_i = alpha_max

	# $i \leftarrow 1$
	i = 1

	# pre-eval $\phi(0)$
	phi_0 = fobj.f(x_k)

	# from (A.16) we have $\phi'(\alpha) = \nabla f(x_k + \alpha p_k)^T p_k$
	# so $\phi'(0) = \nabla f_k^Tp_k$
	phi_prime_0 = dot(fobj.grad_f(x_k), p_k)

	phi_alpha_i_minus_1 = phi_0

	# intelligent values for these found on pg 37-38
	c_1 = 10e-4
	c_2 = 0.99

	while 1:
		# Evaluate $\phi(\alpha_i)$
		phi_alpha_i = fobj.f(x_k + alpha_i * p_k)

		# if $\phi(\alpha_i) > \phi(0) + c_1\alpha_i\phi'(0)$ or $[\phi(\alpha_i) \geq \phi(\alpha_{i-1})$ and $i > 1]$
		if phi_alpha_i > phi_0 + c_1*alpha_i*phi_prime_0 or (phi_alpha_i >= phi_alpha_i_minus_1 and i > 1):
			# $\alpha_\ast \leftarrow$ zoom$(\alpha_{i-1}, \alpha_i)$ and stop;
			return zoom(fobj, x_k, p_k, alpha_i_minus_1, alpha_i, phi_0, phi_prime_0, c_1, c_2)

		# Evaluate $\phi'(\alpha_i) = \nabla f(x_k + \alpha_i p_k)^Tp_k$
		phi_prime_alpha_i = dot(fobj.grad_f(x_k + alpha_i*p_k), p_k)

		# if $|\phi'(a_i)| \leq -c_2\phi'(0)$
		if abs(phi_prime_alpha_i) <= -c_2*phi_prime_0:
			# set $\alpha_\ast \leftarrow \alpha_i$ and stop;
			return alpha_i

		# if $\phi'(\alpha_i) \geq 0$
			# set $\alpha_\ast \leftarrow$ zoom$(\alpha_i, \alpha_{i-1})$ and stop
			return zoom(fobj, x_k, p_k, alpha_i, alpha_i_minus_1, phi_0, phi_prime_0, c_1, c_2)

		# advance our variables
		alpha_i_minus_1 = alpha_i
		phi_alpha_i_minus_1 = phi_alpha_i

		# Choose $\alpha_{i+1} \in (\alpha_i, \alpha_{max})$
		alpha_i = min(alpha_max, 1.1*alpha_i)

		# $i \leftarrow i + 1$
		i += 1


<>:18: SyntaxWarning: invalid escape sequence '\p'
<>:18: SyntaxWarning: invalid escape sequence '\p'
/tmp/ipython-input-203587291.py:18: SyntaxWarning: invalid escape sequence '\p'
  phi_0 is $\phi(0)$, a floating point scalar


## BFGS Line Search

In [13]:

def BFGSLineSearch(x_0, alpha_0, f_alpha_k, fobj, epsilon, H_0):
	"""
	An implementation of the modified BFGS algorithm
	from Nocedal and Wright page 198.

	Inputs:
		x_0 is a Numeric array giving the initial value $x_0$

		alpha_0 is the $\alpha_0$, the inital value for $\alpha$.

		f_alpha_k is the line search function, it takes a fobj,
			x_k, p_k, and alpha_max and returns an alpha value
			for the current iteration.  Recommend StrongWolfe above.

		fobj is the objective function object

		epsilon is the tolerance value of $\epsilon$, the function
			returns when $\|\nabla f_k\| > \epsilon$

		H_0 is the inital inverse Hessian estimate $H_0$

	Outputs:
		Returns the computed value of the stationary point of $f$, $x^\ast$,
			within tolerance $\epsilon$

	Side Effects:
		Reports to stdout at each iteration the value of $\alpha_k$,
			$x_k$, the number of function evaluations, and
			if the product $y_k^Ts_k$ is positive.
	"""
	# $k \leftarrow 0$
	k = 0

	x_k = x_0
	H_k = H_0

	# print out our table header
	print ('\n\nBFGS Line Search on %s' \
		% fobj.descriptive_name())
	print ('alpha_0 = %f, x_0 = %s' % (alpha_0, x_0))
	print ('iteration\talpha_k\t\tx_k\t\t\t\tf_evals\tyk*sk positive?')

	# while $\|\nabla f_k\|$
	while sqrt(dot(fobj.grad_f(x_k), fobj.grad_f(x_k))) > epsilon:
		# $p_k = -H_k \nabla f_k$
		p_k = -1.0 * matrixmultiply(H_k, fobj.grad_f(x_k))

		# apply the line search technique supplied
		alpha_k = f_alpha_k(fobj, x_k, p_k, alpha_0)

		# $x_{k+1} = x_k + \alpha p_k$
		x_k_plus_1 = x_k + (alpha_k * p_k)

		# $s_k = x_{k+1} - x_k$
		s_k = x_k_plus_1 - x_k

		# $y_k = \nabla f_{k+1} - \nabla f_k$
		y_k = fobj.grad_f(x_k_plus_1) - fobj.grad_f(x_k)

		# $\rho_k = (y_k^T s_k)^{-1}$
		rho_k = 1.0 / dot(y_k, s_k)

		# $H_{k+1} = (I - \rho_k s_k y_k^T) H_k (I - \rho_k y_k s_k^T) + p_k s_k s_k^T$
		H_k_plus_1 = matrixmultiply(\
			matrixmultiply(identity(x_0.shape[0]) - \
				rho_k*matrixmultiply(\
				s_k, transpose(y_k)), H_k),\
			identity(x_0.shape[0]) \
			 - rho_k*matrixmultiply(y_k, transpose(s_k))) \
			+ matrixmultiply(matrixmultiply(p_k, s_k), \
					transpose(s_k))

		# $k \leftarrow k+1$
		k = k + 1

		# advance the values
		x_k = x_k_plus_1
		H_k = H_k_plus_1

		# check and make sure that $y_k^Ts_k$ is positive
		if dot(y_k, s_k) > 0:
			y_k_s_k_positive = 'yes'
		else:
			y_k_s_k_positive = 'no'

		# print out a line of the table
		print('%d\t\t%f\t%s\t%d\t%s' \
			% (k, alpha_k, x_k, fobj.eval_count(), y_k_s_k_positive))

		sys.stdout.flush()

	return x_k


<>:17: SyntaxWarning: invalid escape sequence '\e'
<>:17: SyntaxWarning: invalid escape sequence '\e'
/tmp/ipython-input-1516872251.py:17: SyntaxWarning: invalid escape sequence '\e'
  epsilon is the tolerance value of $\epsilon$, the function


## Unit tests for Line Search

In [14]:
#####################
# unit testing code #
#####################

class BFGSTestCase(unittest.TestCase):
	def testDescentFunction(self):
		# Generate a random number from a uniform distribution between -100 and +100
		# loc=-100, scale= +100 - (-100) = +200
		a = uniform.rvs(loc=-100.0, scale=200.0, size=1)[0]
		b = uniform.rvs(loc=-100.0, scale=200.0, size=1)[0]
		x_0 = array((a, b))
		alpha_0 = 1.0

		f1 = testObjFunc()

		x_star = BFGSLineSearch(x_0, alpha_0, \
					StrongWolfe,
					f1, \
					9.9e-13, \
					linalg.inv(f1.hessian_f(x_0)))
		assert np.allclose(x_star, array((0.0, 0.0)))

		# Generate a random number from a uniform distribution between -100 and +100
		# loc=-100, scale= +100 - (-100) = +200
		a = uniform.rvs(loc=-100.0, scale=200.0, size=1)[0]
		b = uniform.rvs(loc=-100.0, scale=200.0, size=1)[0]
		x_0 = array((a, b))
		alpha_0 = 1.0

		f2 = testSecondFunc()

		x_star = BFGSLineSearch(x_0, alpha_0, \
					StrongWolfe, \
					f2, \
					9.9e-13, \
					linalg.inv(f2.hessian_f(x_0)))
		assert np.allclose(x_star, array((1.0, 1.0)))

class SteepestDescentTestCase(unittest.TestCase):
	def testDescentFunction(self):
		# Generate a random number from a uniform distribution between -100 and +100
		# loc=-100, scale= +100 - (-100) = +200
		a = uniform.rvs(loc=-100.0, scale=200.0, size=1)[0]
		b = uniform.rvs(loc=-100.0, scale=200.0, size=1)[0]
		x_0 = array((a, b))
		alpha_0 = 1.0

		x_star = SteepestDescentLineSearch(x_0, alpha_0, \
				  BacktrackingAlpha, \
				  testObjFunc(), \
				  9.9e-13)
		assert np.allclose(x_star, array((0.0, 0.0)))

class NewtonsMethodTestCase(unittest.TestCase):
	def testDescentFunction(self):
		# Generate a random number from a uniform distribution between -100 and +100
		# loc=-100, scale= +100 - (-100) = +200
		a = uniform.rvs(loc=-100.0, scale=200.0, size=1)[0]
		b = uniform.rvs(loc=-100.0, scale=200.0, size=1)[0]
		x_0 = array((a, b))
		alpha_0 = 1.0

		x_star = NewtonsMethodLineSearch(x_0, alpha_0, \
				BacktrackingAlpha, \
				testObjFunc(), \
				9.9e-13)
		assert np.allclose(x_star, array((0.0, 0.0)))

		# Generate a random number from a uniform distribution between -100 and +100
		# loc=-100, scale= +100 - (-100) = +200
		a = uniform.rvs(loc=-100.0, scale=200.0, size=1)[0]
		b = uniform.rvs(loc=-100.0, scale=200.0, size=1)[0]
		x_0 = array((a, b))
		alpha_0 = 1.0

		x_star = NewtonsMethodLineSearch(x_0, alpha_0, \
				 BacktrackingAlpha, \
				 testSecondFunc(), \
				 9.9e-13)
		assert np.allclose(x_star, array((1.0, 1.0)))

In [15]:
unittest.main(argv=['first-arg-is-ignored'], exit=False)



BFGS Line Search on test function
alpha_0 = 1.000000, x_0 = [-48.1644201  -78.60728299]
iteration	alpha_k		x_k				f_evals	yk*sk positive?
1		1.000000	[0. 0.]	5	yes


BFGS Line Search on a second test function
alpha_0 = 1.000000, x_0 = [-10.24761549  19.55442117]
iteration	alpha_k		x_k				f_evals	yk*sk positive?
1		1.000000	[1. 1.]	5	yes


.



Newton's Method Line Search on test function
alpha_0 = 1.000000, x_0 = [ 69.37714911 -30.60070131]
iteration alpha_k   x_k       f evals   
         1    1.0000[0. 0.]         4


Newton's Method Line Search on a second test function
alpha_0 = 1.000000, x_0 = [-51.57997261  47.00097139]
iteration alpha_k   x_k       f evals   
         1    1.0000[1. 1.]         4


...........

1.0

--- Debugging testProblem4_1Hessian ---
Input x: [0. 0.]
Calculated Hessian: [[42. -0.]
 [-0. 20.]]
Expected Hessian: [[42  0]
 [ 0 20]]
Input x: [1. 0.]
Calculated Hessian: [[ 42. -40.]
 [-40.  20.]]
Expected Hessian: [[ 42 -40]
 [-40  20]]
Input x: [0. 1.]
Calculated Hessian: [[ 2. -0.]
 [-0. 20.]]
Expected Hessian: [[ 2  0]
 [ 0 20]]
Input x: [1. 1.]
Calculated Hessian: [[  2. -40.]
 [-40.  20.]]
Expected Hessian: [[  2 -40]
 [-40  20]]


Steepest Descent Line Search on test function
alpha_0 = 1.000000, x_0 = [ 36.35211959 -79.1019902 ]
iteration	alpha_k		x_k				f evals
1		0.500000	[0. 0.]	4


.
----------------------------------------------------------------------
Ran 13 tests in 0.040s

OK


# Trust Region

## bisection(.): Root finding within a and b.

In [16]:
def bisection(fobj, a, b, xtol=9.9e-13, maxiter=1000):
	"""
	Find the root using the bisection method.  I copied part of the
	function signature from scipy.optimize.bisect(), which is
	scipy's version of this function

	Inputs:
		fobj:  object with member f, a one dimensional function taking a number

		a:  Number, one end of the bracketing interval

		b:  Number, other end of the bracketing interval

		xtol: Number, allowable tolerance on f for solution
			should be $\geq$ 0

		maxiter: Number, maximum of iterations allowable

	Returns:
		The found solution, or NaN if no solution was found
	"""
	iter_count = 0
	while True:
		x_mid = (b + a) / 2.0
		f_x_mid = fobj.f(x_mid)
		iter_count += 1
		if sign(f_x_mid) != sign(fobj.f(a)):
			b = x_mid
		elif sign(f_x_mid) != sign(fobj.f(b)):
			a = x_mid
		else:
			return NaN
		maxiter -= 1
		if math.fabs(f_x_mid) < xtol or maxiter == 0:
			break

	if maxiter == 0:
		x_mid = NaN

	return x_mid

<>:15: SyntaxWarning: invalid escape sequence '\g'
<>:15: SyntaxWarning: invalid escape sequence '\g'
/tmp/ipython-input-1902976910.py:15: SyntaxWarning: invalid escape sequence '\g'
  should be $\geq$ 0


## Tau Minimizer

In [17]:
class TauMinimizer:
	"""
	A small class that defines a function $f(\tau)$ with

	$f(\tau) = (p_j + \tau d_j)^T(p_j + \tau d_j) - \Delta^2$
	"""
	def __init__(self, p_j, d_j, Delta):
		"""
		Here in the constructor we define the values for
		$p_j$, $d_j$, and $\Delta$ for use in our function.
		"""
		self.p_j = p_j
		self.d_j = d_j
		self.Delta = Delta

	def f(self, tau):
		"""
		Return the function with the parameters given in the ctor
		evaluated at tau = $\tau$
		"""
		return dot(self.p_j + tau*self.d_j, self.p_j + tau*self.d_j) - self.Delta**2

<>:5: SyntaxWarning: invalid escape sequence '\D'
<>:10: SyntaxWarning: invalid escape sequence '\D'
<>:5: SyntaxWarning: invalid escape sequence '\D'
<>:10: SyntaxWarning: invalid escape sequence '\D'
/tmp/ipython-input-4140902362.py:5: SyntaxWarning: invalid escape sequence '\D'
  $f(\tau) = (p_j + \tau d_j)^T(p_j + \tau d_j) - \Delta^2$
/tmp/ipython-input-4140902362.py:10: SyntaxWarning: invalid escape sequence '\D'
  $p_j$, $d_j$, and $\Delta$ for use in our function.


## CG Steihaug

In [18]:
def CG_Steihaug(fobj, epsilon, Delta, B_k, g):
	"""
	Note that fobj.n should be set to something useful
	before function entry.  Calculates the $p_k$ for the
	next iteration.  Based on algorithm 4.3 on page 75
	of Nocedal and Wright.

	Inputs:
		fobj is the function object, used mostly for n

		epsilon is the tolerance

		Delta is the trust region size

		B_k is the $B_k$, usually the exact hessian at $x_k$

		g is the gradient of f evaluated at $x_k$

	Outputs:
		returns a tuple with the new $p_k$ and 4 integer values,
		3 of which will be zero and one of which will be one.  They
		are in order:
		tolerance - meaning that the p_j matches the tolernace and is good
		boundary - meaning that p_j was outside our trust region
		negative - meaning that the algorithm encountered negative curvature
		maxiter - meaning that the looped maxed out at n

	Side Effects:
		None
	"""
	# make a $p_j$ with dimension of the fobj
	# set $p_0 = p_j = p_0$
	p_j = zeros((fobj.n))

	# set $r_0 = r_j = g$
	r_j = g

	# set $d_0 = d_j = -r_0$
	d_j = -r_j

	# if $\|r_0\| < \epsilon$
	mag_r_0 = math.sqrt(dot(r_j, r_j))
	if mag_r_0 < epsilon:
		# lucked out on choice of p_0
		return (p_j, 1, 0, 0, 0)

	# for $j = 0, 1, 2, \ldots$ dimension of $B$
	for j in range(0, fobj.n):
		# if $d_j^TB_kd_j \leq 0$
		if MatQuad(d_j, B_k, d_j) <= 0:
			# encountered negative curvature
			# Find $\tau$ such that $p = p_j + \tau d_j$ minimizes
			# $m(p)$ in (4.9) and satisfies $\|p\| = \Delta$
			tm = TauMinimizer(p_j, d_j, Delta)
			tau_1 = bisection(tm, 0.0, 200.0*Delta)
			tau_2 = bisection(tm, 0.0, -200.0*Delta)
			p_1 = p_j + tau_1 * d_j
			p_2 = p_j + tau_2 * d_j
			if (p_1 < p_2):
				return (p_1, 0, 0, 1, 0)
			else:
				return (p_2, 0, 0, 1, 0)

		# $\alpha_j = r_j^Tr_j / d_j^TB_kd_j$
		alpha_j = dot(r_j, r_j) / MatQuad(d_j, B_k, d_j)

		# $p_{j+1} = p_j + \alpha_jd_j$
		p_j_plus_1 = p_j + alpha_j * d_j

		# if $\|p_{j+1}\| \geq \Delta$
		if math.sqrt(dot(p_j_plus_1, p_j_plus_1)) >= Delta:
			# reached trust region boundary
			tm = TauMinimizer(p_j, d_j, Delta)
			tau_1 = bisection(tm, 0.0, 200.0*Delta)
			tau_2 = bisection(tm, 0.0, -200.0*Delta)
			if tau_1 > 0:
				return (p_j + tau_1*d_j, 0, 1, 0, 0)
			else:
				return (p_j + tau_2*d_j, 0, 1, 0, 0)

		# $r_{j+1} = r_j + \alpha_jB_kd_j$
		r_j_plus_1 = r_j + alpha_j*matrixmultiply(B_k, d_j)

		# if $\|r_{j+1}\| < \epsilon\|r_0\|$
		if math.sqrt(dot(r_j_plus_1, r_j_plus_1)) < epsilon*mag_r_0:
			#print 'met stopping test'
			return (p_j_plus_1, 1, 0, 0, 0)

		# $\beta_{j+1} = r_{j+1}^Tr_{j+1}/r_j^Tr_j$
		beta_j_plus_1 = dot(r_j_plus_1, r_j_plus_1) / dot(r_j, r_j)

		# $j_{j+1} = r_{j+1} + \beta_{j+1}d_j$
		d_j_plus_1 = -r_j_plus_1 + beta_j_plus_1*d_j

		# advance all the plus ones
		p_j = p_j_plus_1
		r_j = r_j_plus_1
		d_j = d_j_plus_1

	# if we get here we maxed out of the loop
	# maxed out on loop
	return (p_j, 0, 0, 0, 1)

## Local quadratic model

In [19]:
def localmodel(fobj, x_k, p):
	"""
	Returns


		$f(x_k) + \nabla f(x_k)^T p + \frac{1}{2}p^T\nabla^2f(x_k) p$

		with $B$ the exact hessian
	"""
	return fobj.f(x_k) + dot(fobj.grad_f(x_k), p) \
		+ 0.5*MatQuad(p, fobj.hessian_f(x_k), p)


## Trust Region algorithm


In [20]:
def TrustRegion(x_0, Delta_Bar, Delta_0, eta, fobj, maxiter):
	"""
	This is an implementation of algorithm 4.1 in Nocedal
	and Wright, page 68.

	Inputs:
		x_0 is the $x_0$ value, an initial value for $x$

		Delta_Bar is the $\overline{\Delta}$ value,
			the max value for Delta

		Delta_0 is the $\Delta_0$ value, the inital radius
			of the trust region

		eta is the $\eta$ value, the minimum acceptable
			reduction of $f_k$

		fobj is the function object

		maxiter is the maximum number of iterations accepted.

	Outputs:
		Returns the stationary point of $f$, $x^\ast$.

	Side Effects:
		Prints the counts of the different return states of
		CG_Steihaug at the end of the algorithm.
	"""
	x_k = x_0
	Delta_k = Delta_0

	# we keep counters of the exit conditions of CG_Steihaug
	tol = 0
	bound = 0
	neg = 0
	maxed = 0

	for k in range(0, maxiter):
		p_k, t, b, n, m = CG_Steihaug(fobj, 9.9e-13, Delta_k, fobj.hessian_f(x_k), fobj.grad_f(x_k))

		# update the exit condition counters
		tol += t
		bound += b
		neg += n
		maxed += m

		# $\rho_k = \frac{f(x_k) - f(x_k + p_k)}{m_k(0) - m_k(p_k)}$
		rho_k = (fobj.f(x_k) - fobj.f(x_k + p_k)) \
			 / (localmodel(fobj, x_k, zeros((fobj.n))) \
			    - localmodel(fobj, x_k, p_k))

		# precaculate $\|p_k\|$
		mag_p_k = math.sqrt(dot(p_k, p_k))

		# if $\rho_k < \frac{1}{4}$
		if rho_k < 0.25:
			# $\Delta_{k+1} = \frac{1}{4}\|p_k\|$
			Delta_k = 0.25*mag_p_k
		else:
			# if $\rho_k > \frac{3}{4}$ and $\|p_k\| = \Delta_k$
			if rho_k > 0.75 and mag_p_k - Delta_k < 9.9e-13:
				Delta_k = 2*Delta_k
				if Delta_k > Delta_Bar:
					Delta_k = Delta_Bar
		# if $\rho_k > \eta$
		if rho_k > eta:
			# $x_{k+1} = x_k + p_k$
			x_k = x_k + p_k

		# break out if we've found a stationary point within tolerance
		if math.sqrt(dot(fobj.grad_f(x_k), fobj.grad_f(x_k))) < 9.9e-13:
			break

	print ('tolerance met: %d, trust region boundary: %d, negative curvature: %d, maximum iterations: %d' % \
	    (tol, bound, neg, maxed))

	print ('Number of function evaluations: %d' % fobj.eval_count())

	return x_k

<>:9: SyntaxWarning: invalid escape sequence '\o'
<>:9: SyntaxWarning: invalid escape sequence '\o'
/tmp/ipython-input-2800652159.py:9: SyntaxWarning: invalid escape sequence '\o'
  Delta_Bar is the $\overline{\Delta}$ value,


## Unit testing code for trust region

In [21]:
class TrustRegionTestCase(unittest.TestCase):
	def testTrustRegion(self):
    # Generate a random number from a uniform distribution between -100 and +100
		# loc=-100, scale= +100 - (-100) = +200
		a = uniform.rvs(loc=-100.0, scale=200.0, size=1)[0]
		b = uniform.rvs(loc=-100.0, scale=200.0, size=1)[0]
		x_0 = array((a, b))
		alpha_0 = 1.0
		f1 = testObjFunc()

		x_star = TrustRegion(x_0, 10.0, 1.0, 0.9, f1, 1000)
		assert np.allclose(x_star, array((0.0, 0.0)))

    # Generate a random number from a uniform distribution between -100 and +100
		# loc=-100, scale= +100 - (-100) = +200
		a = uniform.rvs(loc=-100.0, scale=200.0, size=1)[0]
		b = uniform.rvs(loc=-100.0, scale=200.0, size=1)[0]
		x_0 = array((a, b))
		alpha_0 = 1.0
		f2 = testSecondFunc()

		x_star = TrustRegion(x_0, 10.0, 1.0, 0.9, f2, 1000)
		assert np.allclose(x_star, array((1.0, 1.0)))

In [22]:
unittest.main(argv=['first-arg-is-ignored'], exit=False)



BFGS Line Search on test function
alpha_0 = 1.000000, x_0 = [ 45.28889838 -92.6107201 ]
iteration	alpha_k		x_k				f_evals	yk*sk positive?
1		1.000000	[0. 0.]	5	yes


BFGS Line Search on a second test function
alpha_0 = 1.000000, x_0 = [-84.72719696  -5.89662547]
iteration	alpha_k		x_k				f_evals	yk*sk positive?
1		1.000000	[1. 1.]	5	yes


.



Newton's Method Line Search on test function
alpha_0 = 1.000000, x_0 = [-46.75507361 -46.4323189 ]
iteration alpha_k   x_k       f evals   
         1    1.0000[0. 0.]         4


Newton's Method Line Search on a second test function
alpha_0 = 1.000000, x_0 = [-23.50542976  83.876756  ]
iteration alpha_k   x_k       f evals   
         1    1.0000[1. 1.]         4


...........

1.0

--- Debugging testProblem4_1Hessian ---
Input x: [0. 0.]
Calculated Hessian: [[42. -0.]
 [-0. 20.]]
Expected Hessian: [[42  0]
 [ 0 20]]
Input x: [1. 0.]
Calculated Hessian: [[ 42. -40.]
 [-40.  20.]]
Expected Hessian: [[ 42 -40]
 [-40  20]]
Input x: [0. 1.]
Calculated Hessian: [[ 2. -0.]
 [-0. 20.]]
Expected Hessian: [[ 2  0]
 [ 0 20]]
Input x: [1. 1.]
Calculated Hessian: [[  2. -40.]
 [-40.  20.]]
Expected Hessian: [[  2 -40]
 [-40  20]]


Steepest Descent Line Search on test function
alpha_0 = 1.000000, x_0 = [-59.58863676 -90.36670183]
iteration	alpha_k		x_k				f evals
1		0.500000	[0. 0.]	4


..
----------------------------------------------------------------------
Ran 14 tests in 0.143s

OK


tolerance met: 1, trust region boundary: 9, negative curvature: 0, maximum iterations: 0
Number of function evaluations: 32
tolerance met: 1, trust region boundary: 12, negative curvature: 0, maximum iterations: 1
Number of function evaluations: 44


# Conjugate Gradient

## Hilbert Matrix Code

In [23]:
import numpy as np

def MakeHilbertMatrix(n):
        """
        Makes a Hilbert matrix of dimension nxn.  The $ij$th component
        of the Hilbert matrix is defined to be:

        $A_{i,j} = 1/(i+j-1)$

        Inputs:
                n is the dimension of the desired Hilbert matrix

        Outputs:
                Returns a Hilbert matrix of dimension nxn

        Side Effects:
                None
        """
        # Use broadcasting to create the matrix more efficiently
        i = np.arange(1, n + 1).reshape(-1, 1) # Column vector from 1 to n
        j = np.arange(1, n + 1)               # Row vector from 1 to n
        # The formula is 1 / (i + j - 1). Broadcasting handles the element-wise creation.
        hilbert_matrix = 1.0 / (i + j - 1)

        return hilbert_matrix

## Conjugate Gradient Algorithm

In [24]:
def ConjugateGradient(x_0, A, b, residual, maxiter):
	"""
	This is an implementation of Algorithm 5.2 defined
	on page 111 of Nocedal and Wright.  It provides an
	estimate for the inverse of a matrix A to approximately
	solve the linear system

	$Ax = b$

	With A an nxn array, x a vector of dimension n, and
	b a solution vector of dimension n.

	Inputs:
		x_0, an inital estimate of the solution $x^\ast$

		A the nxn matrix for which we need an inverse to
			solve the linear system

		b the n dimension vector

		residual is the tolerance allowed on $r_k^Tr_k$

		maxiter is the maximum number of iterations allowable

	Outputs:
		Returns the number of iterations required to reduce
		the error factor $r_k^Tr_k$ below the residual argument

		Also the value $x^\ast$ that it found
	"""
	# $r_0 = r_k \leftarrow Ax_0 - b$
	r_k = matrixmultiply(A, x_0) - b

	# $p_0 = p_k \leftarrow -r_0$
	p_k = -r_k

	# $k \leftarrow 0$
	k = 0
	x_k = x_0

	while math.sqrt(dot(r_k, r_k)) > residual and k < maxiter:
		# $\alpha_k \leftarrow \frac{r_k^Tr_k}{p_k^TAp_k}$
		alpha_k = dot(r_k, r_k) / MatQuad(p_k, A, p_k)

		# $x_{k+1} \leftarrow x_k + \alpha_kp_k$
		x_k = x_k + alpha_k*p_k

		# $r_{k+1} \leftarrow r_k + \alpha_kAp_k$
		r_k_plus_1 = r_k + alpha_k*matrixmultiply(A, p_k)

		# $\beta_{k+1} \leftarrow \frac{r_{k+1}^Tr_{k+1}}{r_k^Tr_k}$
		beta_k_plus_1 = dot(r_k_plus_1, r_k_plus_1) / dot(r_k, r_k)

		# $p_{k+1} \leftarrow -r_{k+1} + \beta_{k+1}p_k$
		p_k = -1.0*r_k_plus_1 + beta_k_plus_1*p_k

		# $k \leftarrow k+1$
		k += 1

		# advance $r_k$
		r_k = r_k_plus_1

	return (k, x_k)


## GetCGIterations(n) for Hilbert Matrix

In [25]:
def GetCGIterations(n):
	"""
	Returns the number of iterations required to solve the
	linear system $Ax = b$ of dimension $n$ using the
	ConjugateGradient algorithm defined above, and with
	$A$ as an nxn Hilbert matrix.

	Inputs:
		n, the dimension of the system

	Outputs:
		The number of iterations CG required to solve the
		system

	Side Effects:
		None
	"""
	A = MakeHilbertMatrix(n)
	b = np.ones(n)
	x_0 = np.zeros(n)
	iters, x_star = ConjugateGradient(x_0, A, b, 10e-6, 10000)
	return iters

## Unit testing for Conjugate Gradient Code

In [26]:
#####################
# unit testing code #
#####################

class ConjugateGradientTestCase(unittest.TestCase):
	def testConjugateGradient(self):
		print('testing ConjugateGradient')

		# CG should solve identity system in one iteration
		A = identity(300)
		x_0 = np.zeros(300)
		b = np.ones(300)

		(iters, x_star) = ConjugateGradient(x_0, A, b, 0, 300)
		assert iters == 1

		remains = matrixmultiply(A, x_star) - b
		assert dot(remains, remains) == 0

In [27]:
unittest.main(argv=['first-arg-is-ignored'], exit=False)



BFGS Line Search on test function
alpha_0 = 1.000000, x_0 = [-43.7544647  -47.04638537]
iteration	alpha_k		x_k				f_evals	yk*sk positive?
1		1.000000	[0. 0.]	5	yes


BFGS Line Search on a second test function
alpha_0 = 1.000000, x_0 = [75.07556577 83.26035177]
iteration	alpha_k		x_k				f_evals	yk*sk positive?
1		1.000000	[1. 1.]	5	yes


..

testing ConjugateGradient


Newton's Method Line Search on test function
alpha_0 = 1.000000, x_0 = [60.74464904 98.26163572]
iteration alpha_k   x_k       f evals   
         1    1.0000[0. 0.]         4


Newton's Method Line Search on a second test function
alpha_0 = 1.000000, x_0 = [39.70364182 74.51731126]
iteration alpha_k   x_k       f evals   
         1    1.0000[1. 1.]         4


...........

1.0

--- Debugging testProblem4_1Hessian ---
Input x: [0. 0.]
Calculated Hessian: [[42. -0.]
 [-0. 20.]]
Expected Hessian: [[42  0]
 [ 0 20]]
Input x: [1. 0.]
Calculated Hessian: [[ 42. -40.]
 [-40.  20.]]
Expected Hessian: [[ 42 -40]
 [-40  20]]
Input x: [0. 1.]
Calculated Hessian: [[ 2. -0.]
 [-0. 20.]]
Expected Hessian: [[ 2  0]
 [ 0 20]]
Input x: [1. 1.]
Calculated Hessian: [[  2. -40.]
 [-40.  20.]]
Expected Hessian: [[  2 -40]
 [-40  20]]


Steepest Descent Line Search on test function
alpha_0 = 1.000000, x_0 = [-18.83171102 -31.50826866]
iteration	alpha_k		x_k				f evals
1		0.500000	[0. 0.]	4


.

tolerance met: 1, trust region boundary: 10, negative curvature: 0, maximum iterations: 0
Number of function evaluations: 35


.
----------------------------------------------------------------------
Ran 15 tests in 0.130s

OK


tolerance met: 1, trust region boundary: 16, negative curvature: 0, maximum iterations: 0
Number of function evaluations: 53


# Problems in the book

In [28]:
def problem_3_1():
	x_0 = array((1.2, 1.2))
	alpha_0 = 1

	# does not converge quickly, even after 5500 iterations
	# it only has $x^\ast$ calculated to 4 decimal places
	x_star = SteepestDescentLineSearch(x_0, alpha_0, \
			BacktrackingAlpha, \
			Rosenbrock(), \
			9.9e-2)

	x_0 = array((1.2, 1.2))
	alpha_0 = 1

	x_star = NewtonsMethodLineSearch(x_0, alpha_0,\
			   BacktrackingAlpha, \
			   Rosenbrock(), \
			   9.9e-13)

	x_0 = array((-1.2, 1))
	alpha_0 = 1

	# again, doesn't converge in a tractable amount of time
	x_star = SteepestDescentLineSearch(x_0, alpha_0, \
			BacktrackingAlpha,\
			Rosenbrock(), \
			9.9e-2)

	x_0 = array((-1.2, 1))
	alpha_0 = 1

	x_star = NewtonsMethodLineSearch(x_0, alpha_0,\
			   BacktrackingAlpha, \
			   Rosenbrock(), \
			   9.9e-13)

def problem_3_9():
	x_0 = array((1.2, 1.2))
	alpha_0 = 1
	r = Rosenbrock()

	x_star = BFGSLineSearch(x_0, alpha_0, \
			StrongWolfe, \
			r, \
			9.9e-13, \
			linalg.inv(r.hessian_f(x_0)))

	x_0 = array((-1.2, 1))

	x_star = BFGSLineSearch(x_0, alpha_0, \
				StrongWolfe,
				r,
				9.9e-13, \
				linalg.inv(r.hessian_f(x_0)))

def problem_4_1():
	# we need the solution of $p_k$ as a function of $\Delta$
	print ('x = (0, -1)')
	print ('Delta\tp_k')

	p = Problem4_1(2)
	B_k = p.hessian_f(array((0, -1)))
	epsilon = 9.9e-13
	g = p.grad_f(array((0, -1)))
	for Delta in arange(0, 2, 0.01):
		p_k, t, b, n, m = CG_Steihaug(p, epsilon, Delta, B_k, g)
		print ('%f\t%s' % (Delta, p_k))

	print ('x = (0, 0.5)')
	print ('Delta\tp_k')

	p = Problem4_1(2)
	B_k = p.hessian_f(array((0, 0.5)))
	epsilon = 9.9e-13
	g = p.grad_f(array((0, 0.5)))
	for Delta in arange(0, 2, 0.01):
		p_k, t, b, n, m = CG_Steihaug(p, epsilon, Delta, B_k, g)
		print ('%f\t%s' % (Delta, p_k))



def problem_4_3():
	maxiter = 10000

	print ('***** Trust Region ***** for n = 10:')
	for i in range(0, 1):
		x_0 = []
		for j in range(0, 10):
			x_0.append(uniform.rvs(loc=-100.0, scale=200.0, size=1)[0])
		x_0 = transpose(array((x_0)))

		print ('x_0: %s' % x_0)

		x_star = TrustRegion(x_0, 10.0, 1.0, 0.1, Problem4_3(10), 1000)

		print ('x_star: %s' % x_star)

	print ('***** Trust Region ***** for n = 50:')
	for i in range(0, 1):
		x_0 = []
		for j in range(0, 50):
			x_0.append(stats.rand.uniform(0, 2)[0])
		x_0 = transpose(array((x_0)))

		print ('x_0: %s' % x_0)

		x_star = TrustRegion(x_0, 10.0, 1.0, 0.1, Problem4_3(50), 1000)

		print ('x_star: %s' % x_star)


def problem_5_1():
	print ('n\titerations')
	print ('5\t%s' % GetCGIterations(5))
	print ('8\t%s' % GetCGIterations(8))
	print ('12\t%s' % GetCGIterations(12))
	print ('20\t%s' % GetCGIterations(20))
	print ('100\t%s' % GetCGIterations(100))

In [29]:
problem_3_1()



Steepest Descent Line Search on the Rosenbrock function
alpha_0 = 1.000000, x_0 = [1.2 1.2]
iteration	alpha_k		x_k				f evals
1		0.000977	[1.08710938 1.246875  ]	13
2		0.000977	[1.11457059 1.23416637]	25
3		0.000977	[1.11081971 1.23574864]	37
4		0.000977	[1.11139655 1.23539157]	49


Newton's Method Line Search on the Rosenbrock function
alpha_0 = 1.000000, x_0 = [1.2 1.2]
iteration alpha_k   x_k       f evals   
         1    1.0000[1.19591837 1.43020408]         4
         2    0.5000[1.09828449 1.19668813]         8
         3    1.0000[1.06448816 1.13199285]        11
         4    1.0000[1.01199212 1.02137221]        14
         5    1.0000[1.00426109 1.00848056]        17
         6    1.0000[1.00005033 1.00008294]        20
         7    1.0000[1.00000018 1.00000035]        23
         8    1.0000[1. 1.]        26


Steepest Descent Line Search on the Rosenbrock function
alpha_0 = 1.000000, x_0 = [-1.2  1. ]
iteration	alpha_k		x_k				f evals
1		0.000977	[-0.98945313  1.0859375 

In [30]:
# problem_3_9()

In [31]:
problem_4_1()

x = (0, -1)
Delta	p_k


NameError: name 'NaN' is not defined

In [32]:
problem_4_3()

***** Trust Region ***** for n = 10:
x_0: [ 73.35215072 -68.78325659 -95.13783805  16.12521385  17.25572796
 -68.57577558 -56.22698958  23.42134627  32.42071576 -71.5333623 ]


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [33]:
problem_5_1()

n	iterations
5	6
8	19
12	20
20	34
100	112
